In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Training Shadow Model

In [ ]:
import pickle
import torch

DATA_PATH = '/content/drive/MyDrive/amlm/pickle/cifar10/mobilenetv2/shadow.p'
# Change the DATA_PATH to your local pickle file path

device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

with open(DATA_PATH, "rb") as f:
    dataset = pickle.load(f)

dataloader = torch.utils.data.DataLoader(
    dataset, batch_size=64, shuffle=False, num_workers=2)

for batch_idx, (img, label) in enumerate(dataloader):
    img = img.to(device)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split


In [ ]:
class CustomDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image, label = self.data[idx]
        if self.transform:
            image = self.transform(image)
        return image, label


### Using shadow.p dataset for training shadow model

In [ ]:
train_data, test_data = train_test_split(dataset, test_size=0.36, random_state=42)

transform = transforms.Compose([
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = CustomDataset(train_data, transform=transform)
test_dataset = CustomDataset(test_data, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=40, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=40, shuffle=False)


In [ ]:
model = models.mobilenet_v2(pretrained=True)
model.classifier[1] = nn.Linear(model.classifier[1].in_features, 10)  # 10 classes in CIFAR


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth
100%|██████████| 13.6M/13.6M [00:00<00:00, 172MB/s]


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)


In [ ]:
print(len(train_loader))

480


Training of shadow model

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

num_epochs = 40

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")

print("Finished Training")


Epoch 1, Loss: 1.365428615361452
Epoch 2, Loss: 0.9551601548989613
Epoch 3, Loss: 0.8036066859960556
Epoch 4, Loss: 0.68066303661714
Epoch 5, Loss: 0.5830749422932665
Epoch 6, Loss: 0.5116408750414848
Epoch 7, Loss: 0.44117229074860614
Epoch 8, Loss: 0.3809711736937364
Epoch 9, Loss: 0.3300514398763577
Epoch 10, Loss: 0.2903782879623274
Epoch 11, Loss: 0.2765183149836957
Epoch 12, Loss: 0.2373372172936797
Epoch 13, Loss: 0.20635938873747364
Epoch 14, Loss: 0.18670996157452463
Epoch 15, Loss: 0.17717469475076844
Epoch 16, Loss: 0.15845651519678844
Epoch 17, Loss: 0.1482510140844776
Epoch 18, Loss: 0.13386676408311662
Epoch 19, Loss: 0.13857179802531996
Epoch 20, Loss: 0.1198943151345399
Epoch 21, Loss: 0.10567103516950738
Epoch 22, Loss: 0.11564747688632147
Epoch 23, Loss: 0.10444723147763095
Epoch 24, Loss: 0.1048007298920614
Epoch 25, Loss: 0.10688692073648175
Epoch 26, Loss: 0.09269695453112944
Epoch 27, Loss: 0.10190789572176678
Epoch 28, Loss: 0.08802806548386191
Epoch 29, Loss: 0.

Evaluating shadow model

In [ ]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total}%")


Test Accuracy: 77.38888888888889%


In [ ]:
torch.save(model, 'amlm_shadow_task1_model.pth')


In [ ]:
from google.colab import files

files.download('amlm_shadow_task1_model.pth')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Running the model on train and test dataset, to get confidence for memebers and non-members respectively

### Generating attack model dataset

Non-member confidence

In [ ]:
import torch.nn.functional as F
attack_model_dataset = []
cnt = 0
with torch.no_grad():
    cnt=0
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        probabilities = F.softmax(outputs, dim=1)
        for i in range(len(images)):
          attack_model_dataset.append(probabilities[i].tolist()+[0]) #0 because it is test data, non-member for the model





In [ ]:
print(attack_model_dataset[0])

[0.0003530642716214061, 3.8236740351749177e-07, 3.568402746623178e-07, 3.3166951851626436e-08, 3.37712187103989e-08, 1.2193202891808141e-08, 2.3274431271147478e-08, 9.798864994081669e-06, 0.9996359348297119, 3.811935869180161e-07, 0]


Member confidence

In [ ]:
cnt = 0
with torch.no_grad():
    cnt=0
    for images, labels in train_loader:

        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        probabilities = F.softmax(outputs, dim=1)
        for i in range(len(images)):
          attack_model_dataset.append(probabilities[i].tolist()+[1]) #1 because it is train data, member for the model




In [ ]:
print(len(attack_model_dataset))

30000


In [ ]:
import pickle
with open('/content/drive/MyDrive/amlm/attack_model_dataset.pkl','wb') as f:
  pickle.dump(attack_model_dataset, f)

In [ ]:
with open('/content/drive/MyDrive/amlm/attack_model_dataset.pkl', 'rb') as f:
  attack_model_dataset = pickle.load(f)

## Training Attack Model

Use confidences achieved to train the model to output 0 or 1 based on membership. Make a binary classifier.



In [ ]:
class CustomAttackDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        tensor = self.data[idx][:-1]
        label = self.data[idx][-1] #last element is the membership, 0 or 1
        return torch.tensor(tensor), label


attack_dataset = CustomAttackDataset(attack_model_dataset)


In [ ]:
class BinaryClassifier(nn.Module):
    def __init__(self):
        super(BinaryClassifier, self).__init__()
        self.fc1 = nn.Linear(10, 64)
        self.fc2 = nn.Linear(64, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 32)
        self.fc5 = nn.Linear(32, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        x = self.fc3(x)
        x = self.fc4(x)
        x = self.fc5(x)
        x = self.sigmoid(x)
        return x

attack_model = BinaryClassifier()


In [ ]:
import pickle
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from torch.utils.data import random_split

train_size = int(0.90 * len(attack_dataset))
test_size = len(attack_dataset) - train_size
train_attack_dataset, test_attack_dataset = random_split(attack_dataset, [train_size, test_size])

train_attack_loader = DataLoader(train_attack_dataset, batch_size=25, shuffle=True)
test_attack_loader = DataLoader(test_attack_dataset, batch_size=25, shuffle=False)


print("train shape:", len(train_attack_dataset))

print("test shape:", len(test_attack_dataset))
#print("y_test shape:", y_test.shape)

train shape: 27000
test shape: 3000


In [ ]:
print(train_attack_dataset[0])

(tensor([4.4266e-10, 9.9997e-01, 2.2709e-10, 1.7107e-10, 2.3548e-10, 5.5173e-10,
        3.2272e-08, 2.0011e-09, 2.4810e-05, 6.9988e-06]), 0)


Training of attack model

In [ ]:


import torch
import torch.nn as nn
import torch.optim as optim


criterion = nn.BCELoss()
optimizer = optim.Adam(attack_model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
attack_model.to(device)

num_epochs = 70

for epoch in range(num_epochs):
    attack_model.train()
    running_loss = 0.0
    cnt = 0
    for tensors, labels in train_attack_loader:
        cnt += 1
        # print(cnt)
        tensors, labels = tensors.to(device), labels.to(device).float()

        optimizer.zero_grad()
        outputs = attack_model(tensors).squeeze()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()


    scheduler.step()

    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_attack_loader)}")

print("Finished Training")


Epoch 1, Loss: 0.622252416803881
Epoch 2, Loss: 0.5902709840900369
Epoch 3, Loss: 0.5856102641809869
Epoch 4, Loss: 0.580577376264113
Epoch 5, Loss: 0.5783115104944617
Epoch 6, Loss: 0.5765505130644197
Epoch 7, Loss: 0.5753236063928516
Epoch 8, Loss: 0.5740418293685825
Epoch 9, Loss: 0.572089725981156
Epoch 10, Loss: 0.5717996062227974
Epoch 11, Loss: 0.5651165048005404
Epoch 12, Loss: 0.5640823460563466
Epoch 13, Loss: 0.5637708230702966
Epoch 14, Loss: 0.5638546044627826
Epoch 15, Loss: 0.5636139520340495
Epoch 16, Loss: 0.56305145378466
Epoch 17, Loss: 0.5629280604146145
Epoch 18, Loss: 0.5629156594751058
Epoch 19, Loss: 0.5630569788592833
Epoch 20, Loss: 0.5627034647873155
Epoch 21, Loss: 0.5620203222665522
Epoch 22, Loss: 0.561764219927567
Epoch 23, Loss: 0.5617447153561645
Epoch 24, Loss: 0.5616911904403457
Epoch 25, Loss: 0.5616854160196252
Epoch 26, Loss: 0.5616543706644465
Epoch 27, Loss: 0.5616226239612809
Epoch 28, Loss: 0.5616470712754461
Epoch 29, Loss: 0.5615820569296678


In [ ]:
attack_model.eval()
correct = 0
total = 0
with torch.no_grad():
    for tensors, labels in test_attack_loader:
        tensors, labels = tensors.to(device), labels.to(device).float()
        outputs = attack_model(tensors).squeeze()
        predicted = (outputs > 0.5).float()
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total}%")


Test Accuracy: 72.46666666666667%


## Evaluating Attack Model using eval.p
Input data to target model, get confidence. Input confidence to attack model, receive output. Compare to groundtruth value of membership.

In [ ]:
import torch
import torchvision.models as models

MODEL_PATH = '/content/drive/MyDrive/amlm/models/mobilenetv2_cifar10.pth'
# Change the MODEL_PATH to your local model path

device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

target_model = models.mobilenet_v2(num_classes=10).to(device)
# Change num_classes to 200 when you use the Tiny ImageNet dataset

state_dict = torch.load(MODEL_PATH, map_location=device)
target_model.load_state_dict(state_dict['net'])
# Test accuracy
acc = state_dict['acc']
# Training epoch (start from 0)
epoch = state_dict['epoch']

In [ ]:

EVAL_DATA_PATH = '/content/drive/MyDrive/amlm/pickle/cifar10/mobilenetv2/eval.p'
# Change the DATA_PATH to your local pickle file path

device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

with open(EVAL_DATA_PATH, "rb") as f:
    eval_dataset = pickle.load(f)

eval_dataloader = torch.utils.data.DataLoader(
    eval_dataset, batch_size=64, shuffle=False, num_workers=2)

for batch_idx, (img, label, membership) in enumerate(eval_dataloader):
    img = img.to(device)

In [ ]:
print(len(eval_dataset))

200


In [ ]:
import torch.nn.functional as F
target_model.eval()
attack_model_eval_dataset = []
total=0
correct = 0
with torch.no_grad():
    cnt=0
    for images, labels, membership in eval_dataloader:

        images, labels = images.to(device), labels.to(device)
        outputs = target_model(images)
        #print(outputs)
        _, predicted = torch.max(outputs.data, 1)
        probabilities = F.softmax(outputs, dim=1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        for i in range(len(images)):
          attack_model_eval_dataset.append(probabilities[i].tolist()+[membership[i]])

print("Accuracy of target model :", 100*(correct/total))

Accuracy of target model : 85.0


In [ ]:
print(attack_model_eval_dataset[0])

[1.2022669579891954e-06, 5.424391474662116e-06, 2.5650746465544216e-05, 6.006690819049254e-05, 2.4279757781187072e-05, 0.9998295307159424, 1.5265151887433603e-05, 2.6308618544135243e-05, 7.08294192008907e-06, 5.01111844641855e-06, tensor(1)]


In [ ]:
attack_eval_dataset = CustomAttackDataset(attack_model_eval_dataset)

In [ ]:
attack_eval_loader = DataLoader(attack_eval_dataset, batch_size=16, shuffle=False)

##Finding Accuracy based on eval.p

In [ ]:
attack_model.eval()
correct = 0
total = 0
with torch.no_grad():
    for tensors, labels in attack_eval_loader:
        tensors, labels = tensors.to(device), labels.to(device).float()
        outputs = attack_model(tensors).squeeze()
        predicted = (outputs > 0.5).float()
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total}%")

Test Accuracy: 65.0%


## Finding predictions for Test.p

In [ ]:
DATA_PATH = "/content/drive/MyDrive/amlm/pickle/cifar10/mobilenetv2/test.p"
# Change the DATA_PATH to your local pickle file path

device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

with open(DATA_PATH, "rb") as f:
    test_dataset = pickle.load(f)

test_dataloader = torch.utils.data.DataLoader(
    test_dataset, batch_size=64, shuffle=False, num_workers=2)


###Using target model to get confidence

In [ ]:
import torch.nn.functional as F
target_model.eval()
final_test_dataset = []
total=0
correct = 0
with torch.no_grad():
    cnt=0
    for images, labels in test_dataloader:

        images, labels = images.to(device), labels.to(device)
        outputs = target_model(images)
        #print(outputs)
        _, predicted = torch.max(outputs.data, 1)
        probabilities = F.softmax(outputs, dim=1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        for i in range(len(images)):
          final_test_dataset.append(probabilities[i].tolist())

In [ ]:
import torch
from torch.utils.data import Dataset

class FinalTestDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        tensor = self.data[idx]
        return torch.tensor(tensor, dtype=torch.float32)

final_test_dataset = FinalTestDataset(final_test_dataset)
final_test_loader = torch.utils.data.DataLoader(final_test_dataset, batch_size=16, shuffle=False) #Do not shuffle


###Using attack model

In [ ]:
import torch
import numpy as np


attack_model.eval()
predicted_values = []


with torch.no_grad():

    for prob in final_test_loader:

        prob = prob.to(device)
        outputs = attack_model(prob).squeeze()
        predicted = (outputs > 0.5).float()
        predicted_values.extend(predicted.cpu().numpy())

# List of predictions to a NumPy array
predicted_array = np.array(predicted_values)

# Printing the NumPy array
print(predicted_array)

# Saving NumPy array to a file
np.save('predicted_values.npy', predicted_array)


[0. 1. 1. ... 1. 1. 1.]


In [ ]:
import numpy as np

def compare_npy_files(file1, file2):
    # Load arrays from the files
    arr1 = np.load(file1)
    arr2 = np.load(file2)

    # Compare the arrays
    if np.array_equal(arr1, arr2):
        print("Arrays are identical.")
    else:
        print("Arrays are different.")

# Paths to the .npy files
file1_path = "/content/latest_predictions.npy"
file2_path = "/task1_mobilenetv2_cifar10.npy"

# Compare the files
compare_npy_files(file1_path, file2_path)

Arrays are different.
